# Modelling the Word Game Wordle

The following notebook was created by Ken Pierce. The intention here give some practice in identifying and including pre-conditions, post-conditions, and data invariants using assertions in Python. These are fundamental concepts in creation of formal models, and the use of Python assertions allows us to explore them in a familiar setting.

## Wordle

Wordle is a word game created by Josh Wardle (https://twitter.com/powerlanguish); it is now owned and published by the New York Times: https://www.nytimes.com/games/wordle/index.html. The instructions for Wordle are as follows:

* Guess the WORDLE in 6 tries.
* Each guess must be a valid 5 letter word. 
* The color of the tiles will change to show how close your guess was to the word.
* A green letter shows it is in the correct position.
* A yellow letter shows it is in the word but in the wrong position.
* A grey letter shows that it is not in the word in any position.

## Building Models

There is no right or wrong way to construct a programme (or formal specification) from natural language specifications. We need to consider the purpose, which guides the abstraction decisions. The steps however can be broadly broken down as:

1. Analyse the functional behaviour from the requirements
2. Extract a list of possible data types (often from nouns) and functions (often from actions)
3. Create a dictionary by giving explanations to items in the list
4. Sketch out data types
5. Sketch out functions
6. Refine and add restrictions
7. Review and refine

The following will guide us through these steps to define a Wordle game.

## Functional Behaviour and Data Types

From the description above, we can identify the key elements of the instructions to understand the functionality and suggest data types:

* There’s a secret **wordle**
    - It has **five** letters (we can infer this though it is not explicit)
    - The player loses after **six** tries
* The player can make a **guess**
    - Guess must be **five** letters
    - Guess must be **valid** (= real word?)
* The game should output for each letter in a guess:
    - If the letter is in the **correct position**
    - If the letter in the **wrong position** (but in the word)
    - If the letter is **not in the word**

# The Programme

## Setup

Here, we simply include some imports for the later, as well as download a list of valid 5-letter words. Note, if the download  does not work, you can simply define a list of custom list of 5-letter strings, i.e. `WORDS = ["HELLO", ...]`.

In [1350]:
from dataclasses import dataclass   # for defining dataclasses
from enum import Enum               # for defining enumerations
import typing                       # for type hinting
import random                       # for selecting a random answer
from typing import List             # for defining custom typed lists
from urllib.request import urlopen  # to load web data

# grab a list of words
url = 'https://raw.githubusercontent.com/tabatkins/wordle-list/main/words'
WORDS = [word.rstrip().decode('UTF-8').upper() for word in urlopen(url).readlines()]



## Data Types and Contants

Next, we can define some simple data types, and constants that we will need to use, based on the above analysis. Here we define:

* The length of a valid word.
* The maximum number of guesses.
* A `Word` type as an alias of `str`
* An enumeration of the three types of clue (i.e. colours)
* An enumeration for the game state, either in play, won, or lost.

In [1351]:
# constants
WORD_LENGTH = 5 
MAX_GUESSES = 6

# define Word as an alias for string
Word = str

# type enumerating the three possible clue colours
Clue = Enum('Clue', ['GREEN', 'YELLOW', 'GREY'])

# type enumerating the state of the game
Gamestate =  Enum('Gamestate', ['WON', 'LOST', 'PLAYING'])

## Guess

We can now introduce a `Guess` type, which represents a guess after it is processed by the game (i.e. one line of the output). We use the `dataclass' decorator and properties to simplify definition. This includes the word that was guessed, and the clues.

1. There are two invariants to add to the `setter` functions; what are these? Complete the assertions and include an appropriate message for when they fail. These should restrict the values that `word` and `clues` can take. 
2. Create some test values for `Guess` to demonstrate the invariants, i.e.  `g1 = Guess("HELLO", [Clue.YELLOW, ...])`.

In [1352]:
@dataclass
class Guess:
    """
    A class to represent a guess in Wordle, which is a Word 
    plus a clue for each letter.
    """
    word: Word
    clues: List[Clue]

    @property
    def word(self) -> Word:
        return self._word

    @word.setter
    def word(self, word: Word):
        # invariant
        assert len(word) == WORD_LENGTH, \
            f"Invariant violated: len(Guess.word) <> {WORD_LENGTH}"  
        self._word = word

    @property
    def clues(self) -> List[Clue]:
        return self._clues

    @clues.setter
    def clues(self, clues: Clue):
        # invariant
        assert len(clues) == WORD_LENGTH, \
            f"Invariant violated: len(Guess.clues) <> {WORD_LENGTH}"              
        self._clues = clues

    def __repr__(self):
        """
        Custom representation for pretty printing.
        """
        cluestr = [str(self.word[i]) + ": " + \
            self.clues[i].name for i in range(WORD_LENGTH)]
        return f"{self.word}: {cluestr}"       


# Auxiliary Functions

There are multiple ways to specify the game. The suggestion here is to have a main class `Game` that includes the state and top-level functions including `make_guess`. To help define those, we define two auxiliary functions:

* `check_letter`: Compute clue given a character, its position, and word
* `check_guess`: Compute clues given a word and the answer

## check_letter 

This function computes the clue (colour) for a given letter, given a word and the index of the letter in the word. We will start with a naïve implementation:

* If it's in the right place, return green (i.e. `Clue.GREEN`)
* If it’s not there, return grey
* Otherwise, return yellow.

Complete the function in the following way:

3. What pre-conditions should be included? These should restrict the parameters. Add these and an appropriate message. 
4. Complete the implementation based on the sketch above.
5. Include some test cases to check your implementation, e.g. `print(check_letter("S", 0, "STOUT"))`.

In [1353]:
def check_letter(letter: str, index: int, word: Word) -> Clue:
    """
    Given a letter and an index, computes the colour of the blue
    based on the word.
    """
    # pre-condition
    assert 0 <= index < WORD_LENGTH and \
           len(word) == WORD_LENGTH, "pre-check_letter failed."
           
    if word[index] == letter: return Clue.GREEN  
    elif letter not in word: return Clue.GREY
    else: return Clue.YELLOW

## Check Guess

Given a word and the correct answer, this function computes the list of clues. This can be achieved by declaring a local variable, and for each letter in the guess, use `check_letter`; we can use `enumerate(guess)` to get the index.

6. What pre-conditions should be included? These should restrict the parameters. Add these and an appropriate message. 
7. Complete the implementation based on the sketch above.
8. Include some test cases to check your implementation, e.g. `print(check_guess("STAND", "STOUT"))`.

In [1354]:

def check_guess(word: Word, guess: Word) -> List[Clue]:
    """
    Given the answer and a guess, compute the list of 
    clues corresponding to each letter.
    """
    # pre-condition
    assert len(word) == WORD_LENGTH and \
           len(guess) == WORD_LENGTH, "pre-check_guess failed"
           
    clues = []
    for i,letter in enumerate(guess):
        clue = check_letter(letter, i ,word)
        clues.append(clue)
    return clues


# Game State and Top-level Functionality

Here we define a `Game` class; the state is the current wordle (`answer`), the guesses made so far, and the state of the game (playing, won, or lost). Functionlity is included to print a message to the player about the game, and to rest the game when it is over.

## make_guess

The top-level function is `make_guesss` which takes a single word as a parameter. This function should use `check_guesss` to compute the guess and store it in `guesses`; it should then update the game state by checking if the game has been won (the word is exactly the same as the answer), or lost (the player reached the maximum number of guesses).

9. What pre-conditions should be included? These should restrict the parameters and the game state in which a guess is made (e.g. the player should not be able to guess if the game finished). Add these and an appropriate message.
10. Complete the implementation based on the sketch above, and play a game! You can alternate `game.make_guess(...)` and `game.print_state()` to play.

Note, can you spot the problem with the naïve implementation of `check_letter`?

In [1355]:

class Game:
    answer: Word
    guesses: List[Guess]
    gstate: Gamestate

    def __init__(self, answer = None): # changed from always starting as 'STOUT' to always be a random word
        """
        Constructor for game.
        """

        self.answer = answer if answer else random.choice(WORDS)
        self.guesses = []
        self.gstate = Gamestate.PLAYING

    def make_guess(self, word: Word):
        """
        Make a guess at the wordle.
        """
        # make guesses uppercase
        word = word.upper() 

        # pre-condition
        assert self.gstate == Gamestate.PLAYING and \
               len(self.guesses) < MAX_GUESSES and \
               word in WORDS, "pre-guess failed."
               
        self.guesses.append(Guess(word, check_guess(self.answer, word)))
        if word == self.answer: self.gstate = Gamestate.WON
        elif len(self.guesses) == MAX_GUESSES: self.gstate = Gamestate.LOST

    def print_state(self):
        """
        Returns a message to the user based on the current state of the game.
        """
        if self.gstate == Gamestate.WON: return(f"You won! You took {len(self.guesses)} guesses.")
        elif self.gstate == Gamestate.LOST: return(f"You lost! The answer was: {self.answer}.")
        else: return(f"Guess the wordle, you have {MAX_GUESSES - len(self.guesses)} guesses remaining.")

    def game_over(self) -> bool:
        """
        Yields true if the game is over (won or lost), false otherwise.
        """
        return self.gstate == Gamestate.WON or self.gstate == Gamestate.LOST 

    def reset(self):
        """
        Reset the game by picking a new word, clearing the guess, and
        setting the state back to playing.
        """
        # pre-condition
        assert self.game_over(), "Cannot reset, game in play"
        self.answer = random.choice(WORDS)
        self.guesses = []
        self.gstate = Gamestate.PLAYING
   

# Changes Below

## Hint

Now that the base rules and basic gameplay have been established, we can continue to enhance the gameplay experience by providing the player with a hint - if they choose to receive one. As we are given that hint must be a single character between [A-Z], I will make the hint simply reveal a letter they have not tried yet that is currently in the word.


In [1356]:
import re

Hint = str

def hint(guesses: List[Guess], answer: Word) -> Hint:

    # pre-condition
    assert re.match('^[A-Z]{%d}$' % WORD_LENGTH, answer) and \
        answer in WORDS and \
        len(guesses) < MAX_GUESSES, "pre-hint failed."
    
    # array of all letter indices
    grey = list(range(0, WORD_LENGTH))

    assert len(guesses) > 0, "no guesses made yet"
    
    for j in range(0, WORD_LENGTH):
        if guesses[-1].clues[j] == Clue.GREEN:
            # remove indices with a green clue already
            grey.remove(j)
        elif guesses[-1].clues[j] == Clue.YELLOW:
            # removes indices with a yellow clue already
            grey.remove(answer.index(guesses[-1].word[j]))
    
    # set hint to a random currently grey letter
    hint = answer[random.choice(grey) if grey else '*']

    assert re.match('^[A-Z]$', hint)

    return hint


## Duplicate Letters

Our current implementation of the clues considers each letter independently, which means that duplicate letters are not handles correctly. For example, if a player guesses "STOUT" and the correct word is "STAND", the first "T" will be labelled green as intended, but the second will be labeled yellow, when it should be grey. To account for this, we will rewrite check_guess to compare the whole word together, removing check_letter entirely.

In [1357]:
def check_guess(word: Word, guess: Word) -> List[Clue]:
    """
    Given the answer and a guess, compute the list of 
    clues corresponding to each letter.
    """
    # pre-condition, keeping same as before
    assert len(word) == WORD_LENGTH and \
           len(guess) == WORD_LENGTH and \
            guess in WORDS and word in WORDS, "pre-check_guess failed"
    
    # placeholders
    clues = [Clue.GREY for x in range(WORD_LENGTH)]


    # greens first
    for i, letter in enumerate(guess):
        if word[i] == letter:
            clues[i] = Clue.GREEN
            word = word[:i] + '*' + word[i + 1:]
    
    # yellows next, greys remaining are grey
    for i, letter in enumerate(guess):
        # either make the comparison of not green or replace it in guess as well, but a bit more work to do that so just this check
        if letter in word and clues[i] != Clue.GREEN:
            clues[i] = Clue.YELLOW
            # replace one instance of the letter in the answer, in case of duplicates
            word = word.replace(letter, '*', 1)
            
    
    return clues
        

## Hard Mode

Wordle has a "Hard mode" with the following definition: "Any revealed hints must be used in subsequent guesses." This rule further constrains what strings a user can guess with. We will implement this using a method called 'hard_guess', which calls the 'make_guess' method when the constraints are met.

In [1358]:
def hard_guess(self, guess: Word):

    # edge case for first guess
    if len(self.guesses) == 0:
        self.make_guess(guess)
        return None


    recent_clues = self.guesses[-1] # obtain most recent guess info (all previous guesses will have incorporated the hints before)
    required = "*" * WORD_LENGTH
    included = []

    for i, letter in enumerate(recent_clues.word):
        if recent_clues.clues[i] == Clue.GREEN:
            required = required[:i] + letter + required[i + 1:]
        elif recent_clues.clues[i] == Clue.YELLOW:
            included.append(letter)

    # regex statement
    pattern_1 = re.compile("^%s$" % required.replace('*', "[A-Z]"))
    
    # check all yellow letters included
    for letter in guess:
        if letter in included:
            included.remove(letter)

    assert pattern_1.match(guess), "pre-hard_guess failed"
    assert not included

    self.make_guess(guess)
    

# Attach the hard_guess method to Game class
Game.hard_guess = hard_guess

## Gameplay

Now that all of the fundamental gameplay mechanics have been implemented, we need to combine them all into one runnable game. There are two possible ways to do this, either with a CLI that tells the user to enter their guesses and output their clues, or with a GUI that allows the user to visually see the colours on their screen. I decided to go for a GUI using PyGame to challenge myself slightly further, as I have very little experience with it and enjoy trying new things.

### Initialising the Game
So we will start by defining constants and initialising a pygame instance.


In [1359]:
# https://www.pygame.org/docs/
import pygame
import sys

# pygame setup
pygame.init()

# constants
WIDTH, HEIGHT = 600, 600
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
CLOCK = pygame.time.Clock()

GREEN = "#009900"
YELLOW = "#FFFF00"
GREY = "#555555"
BLACK = "#000000"
WHITE = "#FFFFFF"
LIGHT_GREY = "#ADADAD"
COLOR_PRIO = [GREEN, YELLOW, GREY, LIGHT_GREY]

OUTLINE_WIDTH = 5
SPACE = 200

BOX_SIZE = ((WIDTH - SPACE - WORD_LENGTH * OUTLINE_WIDTH * 2)//WORD_LENGTH,
            (HEIGHT - SPACE - MAX_GUESSES * OUTLINE_WIDTH * 2)//MAX_GUESSES)


pygame.display.set_caption("Wordle!")

guess: Word

### Letter

We will now define a letter class. We will treat each input letter separately as a single block that appears on the screen, so that they can change color based off their clues independently of the other letters in the word, similar to how check_letter previously functioned, though this will be for visualisation only.

In [1360]:
class Letter:
    
    # take the letter and its position on the screen and define all variables
    def __init__(self, letter, pos, clue = Clue.GREY, box_size = BOX_SIZE):

        # pre-conditions
        assert re.match("^[A-Z]$", letter) and \
            len(pos) == 2 and \
            OUTLINE_WIDTH - SPACE // 2 <= pos[0] <= WIDTH - OUTLINE_WIDTH and \
            OUTLINE_WIDTH <= pos[1] <= HEIGHT - OUTLINE_WIDTH, "pre-Letter failed"
        
        self.letter = letter
        self.bg_col = "white"
        self.letter_col = "black"

        self.pos = pos
        self.x_pos = pos[0] + SPACE // 2
        self.y_pos = pos[1]
        self.box_size = box_size

        self.font = pygame.font.Font(None, round((self.box_size[0]**2 + box_size[1]**2)**(1/2)))
        self.letter_surface = self.font.render(self.letter, True, self.letter_col)
        self.letter_rect = self.letter_surface.get_rect(center=(self.x_pos + (self.box_size[0] - OUTLINE_WIDTH) // 2,
                                                                self.y_pos + self.box_size[1] // 2))

        self.outline_rect = self.letter_surface.get_rect 
        
        self.clue = clue

    # print just the letter
    def __repr__(self):
        return self.letter
      
    # draw the letter on to the screen
    def draw(self, guess: bool = False):
        pygame.draw.rect(SCREEN, BLACK, (self.x_pos - OUTLINE_WIDTH,
                                         self.y_pos - OUTLINE_WIDTH,
                                         self.box_size[0] + OUTLINE_WIDTH,
                                         self.box_size[1] + OUTLINE_WIDTH))
        pygame.draw.rect(SCREEN, self.clue if guess else WHITE, (self.x_pos,
                                         self.y_pos,
                                         self.box_size[0] - OUTLINE_WIDTH,
                                         self.box_size[1] - OUTLINE_WIDTH))
        
        SCREEN.blit(self.letter_surface, self.letter_rect)
        pygame.display.flip()

    def delete(self):
        pygame.draw.rect(SCREEN, WHITE, (self.x_pos,
                                         self.y_pos,
                                         self.box_size[0] - OUTLINE_WIDTH,
                                         self.box_size[1] - OUTLINE_WIDTH))
        pygame.display.flip()

### Hard Mode Button

We need a button that allows the user to choose whether they want to play on easy mode or hard mode. This will just be a clickable button in the top left corner that controls a Boolean value of True for hard mode enabled, or False for easy mode. Since we only need one, there is no need to make a new class for it, instead I will leave it as a single method.

In [1361]:
def hard_mode_button(hard: bool):
    button_rect = pygame.Rect(10, 10, 80, 30)
    button_col = GREEN if hard else GREY
    button_text = "Hard Mode"
    button_font = pygame.font.Font(None, 20)

    text_surface = button_font.render(button_text, True, WHITE)
    text_rect = text_surface.get_rect(center=button_rect.center)

    pygame.draw.rect(SCREEN, button_col, button_rect)
    SCREEN.blit(text_surface, text_rect)

    return button_rect, hard

### Hint Button

We also need a button that can be clicked by the player to give them the hint we made in the code above. This will be similar to the hard mode button but it will also display the hint below it when clicked (I combined these two methods together).

In [1362]:
def hint_button(clicked: bool = False, hint = None):
    if not clicked:
        y_pos = 10
        col = LIGHT_GREY
        text = "Hint"
    else:
        y_pos = 40
        col = YELLOW
        text = hint

    button_rect = pygame.Rect(WIDTH - 60, y_pos, 50, 30)
    button_col = col
    button_text = text
    button_font = pygame.font.Font(None, 20)

    text_surface = button_font.render(button_text, True, BLACK)
    text_rect = text_surface.get_rect(center=button_rect.center)

    pygame.draw.rect(SCREEN, button_col, button_rect)
    SCREEN.blit(text_surface, text_rect)

    return button_rect

### "Keyboard"

To improve user experience, we need a "keyboard" on the screen so that the player knows the overall current state of every letter; whether they have tried it or not. We will store the values in an array and refresh the keys every time the user makes a guess.

In [1363]:

# create keyboard array for positioning on ui
keyboard = ["QWERTYUIOP", "ASDFGHJKL", "ZXCVBNM"]

# Letter objects for the keyboard
keys = []

# index tracker to interact with correct Letter objects
keys_index = []

def reset_keyboard():
    # make Letter objects for each letter and set them to dark grey (unused)
    for i in range(len(keyboard)):
        for j in range(len(keyboard[i])):
            # some maths to make it fit nicely
            keys.append(Letter(keyboard[i][j], (j*(BOX_SIZE[0]//1.5+2*OUTLINE_WIDTH)+OUTLINE_WIDTH+(WORD_LENGTH*(BOX_SIZE[0]+2*OUTLINE_WIDTH)
                                                                                    - (BOX_SIZE[0]//1.5+2*OUTLINE_WIDTH)*len(keyboard[i]))//2,
                                                i*(BOX_SIZE[1]//1.5+2*OUTLINE_WIDTH)+MAX_GUESSES*BOX_SIZE[1]+2*OUTLINE_WIDTH*MAX_GUESSES+
                                                OUTLINE_WIDTH),
                                                LIGHT_GREY,
                                                (BOX_SIZE[0]//1.5,BOX_SIZE[1]//1.5)))
            keys_index.append(keyboard[i][j])


### Gameplay

We need the game to wait for user inputs, then display them on the screen as they type. The game will wait for an event, or a user input until it receives one, then depending on the input with act accordingly. If the user inputs a backspace, assuming they're currently typing a word, it will remove the most recently input letter. Otherwise it will do nothing. As long as the user has not entered too many letters for the current word, entering a character key will add that letter to the word. Pressing the enter key when the word is full will submit the current options as a guess.

In [1364]:
from tkinter import *
from tkinter import messagebox

def main():
    # set background to white
    SCREEN.fill("white")

    # making sure guess count and max word length are not exceeded
    current_guess = 0
    word_to_guess = ""
    letters = []

    # start the actual wordle game
    game = Game()
    
    # reset keyboard
    reset_keyboard()
    
    # create initial grid
    for y in range(MAX_GUESSES):
        for x in range(WORD_LENGTH):
            # borders
            pygame.draw.rect(SCREEN, BLACK, (x*(BOX_SIZE[0] + 2 * OUTLINE_WIDTH) + SPACE // 2,
                                            y*(BOX_SIZE[1] + 2 * OUTLINE_WIDTH),
                                            BOX_SIZE[0] + OUTLINE_WIDTH,
                                            BOX_SIZE[1] + OUTLINE_WIDTH))
            pygame.draw.rect(SCREEN, WHITE, (x*(BOX_SIZE[0] + 2 * OUTLINE_WIDTH) + OUTLINE_WIDTH + SPACE // 2,
                                            y*(BOX_SIZE[1] + 2 * OUTLINE_WIDTH) + OUTLINE_WIDTH,
                                            BOX_SIZE[0] - OUTLINE_WIDTH,
                                            BOX_SIZE[1] - OUTLINE_WIDTH))

    # add keyboard to screen
    for key in keys:
        key.draw(True)

    # add hard mode button initially set to easy mode and hint button
    button = hard_mode_button(False)
    hint_pressed = hint_button()

    while True:
        
        for event in pygame.event.get():
            # pygame.QUIT event means the user clicked X to close their window
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            # pygame.KEYDOWN means if the user pressed a key on their keyboard
            if event.type == pygame.KEYDOWN:
                # enter key
                if event.key == pygame.K_RETURN:
                    if current_guess == MAX_GUESSES or game.gstate == Gamestate.WON:
                        main()
                    # check length of word and then make guess if correct length
                    if len(word_to_guess) == WORD_LENGTH:
                        success = True
                        if button[1]:
                            try:
                                game.hard_guess(word_to_guess)
                            except AssertionError:
                                # https://stackoverflow.com/questions/41639671/pop-up-message-box-in-pygame
                                Tk().wm_withdraw() #to hide the main window
                                messagebox.showinfo("Invalid","That is not a valid hard-mode word. Try Again.")
                                success = False
                                
                        else:
                            try:
                                game.make_guess(word_to_guess)
                            except AssertionError:
                                Tk().wm_withdraw() #to hide the main window
                                messagebox.showinfo("Invalid","That is not a valid word. Try Again.")
                                success = False

                        if success:
                            # colour changing visible on game not print_state()
                            for i, clue in enumerate(game.guesses[-1].clues):
                                letters[i].clue = GREEN if clue == Clue.GREEN else YELLOW if clue == Clue.YELLOW else GREY
                                letters[i].draw(True)
                                # update keyboard colours

                                index = keys_index.index(letters[i].letter)

                                # do not overwrite repeated letters
                                if COLOR_PRIO.index(letters[i].clue) < COLOR_PRIO.index(keys[index].clue):
                                    keys[index].clue = letters[i].clue
                                    keys[index].draw(True)

                                pygame.display.flip()


                            current_guess += 1
                            word_to_guess = ""
                            letters = []

                            # make sure this works as intended (stopping point), also work on "keyboard"
                            if game.gstate != Gamestate.PLAYING:
                                Tk().wm_withdraw() #to hide the main window
                                messagebox.showinfo("Game Over!", game.print_state())


                # backspace
                elif event.key == pygame.K_BACKSPACE:
                    # check length of word (must be > 0) and delete a letter

                    if len(word_to_guess) > 0:
                        word_to_guess = word_to_guess[:-1]
                        letters[-1].delete()
                        letters.pop()

                else:
                    key_pressed = event.unicode.upper()
                    # check it is a capital letter and word has not exceeded max word length
                    if re.match('[A-Z]', key_pressed) and len(word_to_guess) < WORD_LENGTH:
                        # create letter
                        new_letter = Letter(key_pressed, [len(word_to_guess)*(BOX_SIZE[0] + 2 * OUTLINE_WIDTH) + OUTLINE_WIDTH, 
                                                        current_guess*(BOX_SIZE[1] + 2 * OUTLINE_WIDTH) + OUTLINE_WIDTH])
                        new_letter.draw()
                        
                        letters.append(new_letter)
                        word_to_guess += key_pressed

            # check if the hard mode button is pressed
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1: # left mouse click
                    if button[0].collidepoint(event.pos): # if hard mode button pressed
                        button = hard_mode_button(not button[1])
                    elif hint_pressed.collidepoint(event.pos): # if hint button pressed
                        hint_result = hint(game.guesses, game.answer)
                        hint_button(True, hint_result)

                        # add the hint result to the visible keyboard
                        keys[keys_index.index(hint_result)].clue = YELLOW
                        keys[keys_index.index(hint_result)].draw(True)
                        pygame.display.flip()

        # flip() the display to put your work on screen
        pygame.display.flip()

        CLOCK.tick(60)  # limits FPS to 60

main()

SystemExit: 

## Testing

This section shows all the tests that have been made for each of the methods that require it, with all possible scenarios accounted for. The adjacent comment will describe the expected output of the test.

### Game

In [ ]:
def test_game():
    game = Game("SISSY")
    game.print_state() # 6 guesses remaining
    game.make_guess("SLASH") # SLASH: ['S: GREEN', 'L: GREY', 'A: GREY', 'S: GREEN', 'H: GREY']
    game.print_state() # 5 guesses remaining
    game.make_guess("SISSY") #SISSY: ['S: GREEN', 'I: GREEN', 'S: GREEN', 'S: GREEN', 'Y: GREEN']
    game.print_state() # Word found, game ends

    #game2 = Game("POTATO") # Error, word too long

    game3 = Game("STOUT")
    game3.print_state() # 6 guesses remaining
    game3.make_guess("FRAME") # FRAME: ['F: GREY', 'R: GREY', 'A: GREY', 'M: GREY', 'E: GREY']
    game3.print_state() # 5 guesses remaining
    game3.make_guess("FRAME") # FRAME: ['F: GREY', 'R: GREY', 'A: GREY', 'M: GREY', 'E: GREY']
    game3.print_state() # 4 guesses remaining
    game3.make_guess("FRAME") # FRAME: ['F: GREY', 'R: GREY', 'A: GREY', 'M: GREY', 'E: GREY']
    game3.print_state() # 3 guesses remaining
    game3.make_guess("FRAME") # FRAME: ['F: GREY', 'R: GREY', 'A: GREY', 'M: GREY', 'E: GREY']
    game3.print_state() # 2 guesses remaining
    game3.make_guess("FRAME") # FRAME: ['F: GREY', 'R: GREY', 'A: GREY', 'M: GREY', 'E: GREY']
    game3.print_state() # 1 guesses remaining
    game3.make_guess("FRAME") # FRAME: ['F: GREY', 'R: GREY', 'A: GREY', 'M: GREY', 'E: GREY']
    game3.print_state() # 0 guesses remaining, game lost

test_game()

### Hard Mode

In [ ]:
def test_hard_mode():
    game = Game("HARDS")
    game.print_state() # 6 guesses remaining
    game.hard_guess("WORDS") # WORDS: ['W: GREY', 'O: GREY', 'R: GREEN', 'D: GREEN', 'S: GREEN']
    game.print_state() # 5 guesses remaining
    #game.hard_guess("FRAME") # AssertionError, does not include previous hints


    game2 = Game("WORDS")
    game2.print_state() # 6 guesses remaining
    game2.hard_guess("WORDS")
    game2.print_state() # Game won

    game3 = Game("STEAL")
    game3.print_state() # 6 guesses remaining
    game3.hard_guess("MEALS") # MEALS: ['M: GREY', 'E: YELLOW', 'A: YELLOW', 'L: YELLOW', 'S: YELLOW']
    game3.print_state() # 5 guesses remaining
    #game3.hard_guess("SMEALS") # AssertionError, not a real word
    game3.hard_guess("REALS") # REALS: ['R: GREY', 'E: YELLOW', 'A: YELLOW', 'L: YELLOW', 'S: YELLOW']
    game3.print_state() # 4 guesses remaining
    game3.hard_guess("STEAL") # STEAL: ['S: GREEN', 'T: GREEN', 'E: GREEN', 'A: GREEN', 'L: GREEN']
    game3.print_state() # Game won
test_hard_mode()

### Guess

In [14]:
def test_guess():
    g1 = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW]) # Successful Guess object creation
    
    #g2 = Guess("HELL", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN, Clue.YELLOW]) # Assertion Error
    #g3 = Guess("HELLO", [Clue.YELLOW, Clue.YELLOW, Clue.GREY, Clue.GREEN]) # Assertion Error
    
    return g1

print(test_guess()) # HELLO: ['H: YELLOW', 'E: YELLOW', 'L: GREY', 'L: GREEN', 'O: YELLOW'] (using __repr__)

HELLO: ['H: YELLOW', 'E: YELLOW', 'L: GREY', 'L: GREEN', 'O: YELLOW']


### check_letter

In [131]:
def test_check_letter():
    print(check_letter("S", 0, "STOUT")) #Clue.GREEN
    print(check_letter("S", 3, "STOUT")) #Clue.YELLOW
    print(check_letter("Z", 0, "STOUT")) #Clue.GREY

Clue.GREEN
Clue.YELLOW
Clue.GREY


### Hint

In [25]:
def test_hint():
    game = Game("STOUT")
    print(hint(game.guesses, game.answer)) # Any of the 5 letters at random
    game.make_guess("START") # S, T, T green
    print(hint(game.guesses, game.answer)) # Only O or U at random

test_hint()

T
O
